In [4]:
import os
from os import path
import shutil
import logging
import time
import pandas as pd
import tempfile
import json
import requests
import base64
from modules.data_processing import categorize_files_by_template, pdf_to_image_paths, encode_image, load_dataset
from modules.batch_processing import split_jsonl_file, send_batch_requests
from modules.arrange_conf_md import get_log_directory_img, get_output_directory_img, get_data_paths
from modules.batch_processing import sanitize_filename, generate_custom_id
from modules.model_interaction import parse_retry_after, adaptive_delay
from modules.run_pipeline_md import load_experiment_results
from config import MODEL_gpt_4, prompt_reg_img
from multiprocessing import current_process

### Arrange working dirs

In [5]:
directory = 'vrdu2/registration-form/few_shot-splits/'
# directory = 'VT2/vrdu2/ad-buy-form/few_shot-splits/'

# Arrange logging and output dirs accordıng to model and form type
model = MODEL_gpt_4  
log_directory = get_log_directory_img(directory, model) 
base_output_directory = get_output_directory_img(directory, model)

print("log_directory: ", log_directory)
print("base_output_directory: ", base_output_directory)

# Arrange folder and dataset dirs based on the provided main dir
folder_path, dataset_path, dtype = get_data_paths(directory)

print("folder_path:", folder_path)
print("dataset_path:", dataset_path)
print("dtype:", dtype)

log_file_path = path.join(log_directory, f"experiment_{current_process().pid}.log")
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers=[
                        logging.FileHandler(log_file_path),
                        logging.StreamHandler()
                    ])

# Example logging
logging.info("Logging setup complete.")

2025-02-24 12:59:21,430 - INFO - Logging setup complete.


log_directory:  gpt4_outputs_Image/reg/
base_output_directory:  gpt4_outputs_Image/reg/
folder_path: vrdu2/registration-form/main/pdfs
dataset_path: vrdu2/registration-form/main/dataset.jsonl.gz
dtype: reg


### Create experıment confıguratıons

In [5]:
import itertools

sample_size = 40
level_types = ["STL", "UTL"]

combinations = []
experiment_id = 0

for level_type in level_types:
    combination = {
        'experiment_id': experiment_id,
        'dataset': "vrdu",
        'sample_size': sample_size,
        'level_type': level_type
    }
    combinations.append(combination)
    experiment_id += 1
    
df = pd.DataFrame(combinations)

csv_file_path = path.join(base_output_directory, 'all_experiment_combinations.csv')
df.to_csv(csv_file_path, index=False)

print(f"CSV file has been saved to {csv_file_path}")

CSV file has been saved to gpt4_outputs_Image/reg/all_experiment_combinations.csv


### Run pıpelıne wıth created experıments

In [5]:
def perform_experiment(sample_size, experiment_id, output_file, **kwargs):
    logging.info(f"Starting experiment with ID {experiment_id}")
    print(f"Starting experiment with ID {experiment_id}")
    try:
        level_type = kwargs.get('level_type', 'STL')
        
        template_types = ['Amendment', 'Dissemination', 'Short-Form']
        categorized_files = categorize_files_by_template(os.path.join(directory, 'prompts'))

        output_data = []
        task_counter = 0
        existing_custom_ids = set()
        filename_mapping = {}

        for template_type in template_types:
            files_list = list(categorized_files[level_type][template_type])
            sample_files = files_list[:sample_size]
            
            for filename in sample_files:
                if filename.endswith(".pdf"):
                    start_time = time.time()
                    logging.info(f"Experiment {experiment_id}: Processing file {filename}")

                    success = False
                    retry_count = 0
                    max_retries = 10
                    sanitized_filename, original_filename = sanitize_filename(filename)
                    filename_mapping[sanitized_filename] = original_filename 

                    while not success and retry_count < max_retries:
                        try:
                            pdf_path = os.path.join(folder_path, filename)
                            with tempfile.TemporaryDirectory() as temp_pdf_dir:
                                temp_pdf_path = os.path.join(temp_pdf_dir, filename)
                                shutil.copy(pdf_path, temp_pdf_path)

                                images = pdf_to_image_paths(temp_pdf_path, temp_pdf_dir)
                                if not images:
                                    logging.error(f"No images generated from PDF: {pdf_path}")
                                    break

                                for image_path in images:
                                    custom_id = generate_custom_id(experiment_id, level_type, template_type, sanitized_filename, task_counter)
                                    while custom_id in existing_custom_ids:
                                        task_counter += 1
                                        custom_id = generate_custom_id(experiment_id, level_type, template_type, sanitized_filename, task_counter)
                                    
                                    existing_custom_ids.add(custom_id)
                                    task_counter += 1

                                    if image_path.startswith("http"):
                                        image_url = image_path
                                    else:
                                        image_url = f"data:image/jpeg;base64,{encode_image(image_path)}"

                                    # Prepare the request body for the JSONL file
                                    request_body = {
                                        "model": "gpt-4o",  # or another model you're using
                                        "temperature": 0.0,
                                        "messages": [
                                            {
                                                "role": "system",
                                                "content": "You are an LLM that extracts information from a given image. Provide the extracted values in JSON format.",
                                            },
                                            {
                                                "role": "user",
                                                "content": [
                                                    {"type": "text", "text": prompt_reg_img},
                                                    {
                                                        "type": "image_url",
                                                        "image_url": {"url": image_url}
                                                    }
                                                ]
                                            }
                                        ],
                                    }

                                    task = {
                                        "custom_id": custom_id,
                                        "method": "POST",
                                        "url": "/v1/chat/completions",
                                        "body": request_body,
                                    }

                                    # Write the task to the output JSONL file
                                    with open(output_file, "a") as file:
                                        file.write(json.dumps(task) + "\n")

                                success = True
                                output_data.append({
                                "experiment_id": experiment_id,
                                "model_name": MODEL,
                                "sample_num": len(sample_files),
                                "level_type": level_type,
                                "file_name": original_filename
                            })
                                
                        except requests.exceptions.HTTPError as e:
                            if e.response.status_code == 429:
                                retry_count += 1
                                error_details = e.response.json()
                                retry_after = parse_retry_after(error_details)
                                
                                if retry_after:
                                    logging.warning(f"Rate limit reached. Retrying in {retry_after:.2f} seconds due to rate limit.")
                                    print(f"Rate limit reached. Retrying in {retry_after:.2f} seconds due to rate limit.")
                                    time.sleep(retry_after)
                                    logging.info(f"Waited for {retry_after:.2f} seconds before retrying.")
                                    print(f"Waited for {retry_after:.2f} seconds before retrying.")
                                else:
                                    delay = adaptive_delay(retry_count)
                                    logging.warning(f"No specific retry_after provided. Using adaptive delay of {delay:.2f} seconds.")
                                    print(f"No specific retry_after provided. Using adaptive delay of {delay:.2f} seconds.")
                                    time.sleep(delay)
                            else:
                                logging.error(f"HTTP error while processing file {filename}: {e}")
                                print(f"HTTP error while processing file {filename}: {e}")
                                break  # Exit loop on non-rate limit HTTP errors
                        except Exception as e:
                            logging.error(f"An error occurred while processing file {filename}: {e}")
                            print(f"An error occurred while processing file {filename}: {e}")
                            break  # Exit loop on other errors
    except Exception as e:
        logging.error(f"Error during experiment {experiment_id}: {e}")
        return {}, []
    return filename_mapping, output_data

In [7]:
all_experiment_results = []

def run_experiment(params, output_file):
    sample_size, kwargs, experiment_id = params
    try:
        filename_mapping, output_data = perform_experiment(sample_size, experiment_id, output_file, **kwargs)
        return experiment_id, filename_mapping, output_data
    except Exception as e:
        logging.error(f"An error occurred in experiment {experiment_id}: {e}")
        print(f"An error occurred in experiment {experiment_id}: {e}")
        return experiment_id, {}, []

def test_experiment_serially(output_file):
    all_experiment_mappings = {}
    all_experiment_output_data = {}
    
    for i, params in enumerate(experiment_params):
        try:
            experiment_id, filename_mapping, output_data = run_experiment(params, output_file)
            all_experiment_mappings[experiment_id] = filename_mapping
            all_experiment_output_data[experiment_id] = output_data
            if output_data:
                print(f"Experiment {experiment_id} completed successfully.")
            else:
                print(f"Experiment {experiment_id} did not return any results.")
        except Exception as e:
            print(f"Experiment {experiment_id} resulted in an exception: {e}")
    
    with open("gpt4_outputs_Image/reg/all_filename_mappings.json", "w") as f:
        json.dump(all_experiment_mappings, f, indent=4)
    
    with open("gpt4_outputs_Image/reg/all_output_data.json", "w") as f:
        json.dump(all_experiment_output_data, f, indent=4)

    return all_experiment_mappings, all_experiment_output_data

In [8]:
if __name__ == "__main__":
    experiment_params = [(exp['sample_size'], {'level_type': exp['level_type']}, exp['experiment_id']) for exp in combinations]
    output_file = os.path.join(base_output_directory, 'batch_requests.jsonl')
    all_filename_mappings, all_output_data = test_experiment_serially(output_file)

2024-08-25 14:19:52,477 - INFO - Starting experiment with ID 0
2024-08-25 14:19:52,482 - INFO - Experiment 0: Processing file 19830401_Bermuda Tourism Authority_Amendment_Amendment.pdf


Starting experiment with ID 0


2024-08-25 14:19:53,144 - INFO - Converted and resized image saved to /tmp/tmpn1juz21o/page_1.jpg with final quality 85
2024-08-25 14:19:53,148 - INFO - Converted PDF /tmp/tmpn1juz21o/19830401_Bermuda Tourism Authority_Amendment_Amendment.pdf to images: ['/tmp/tmpn1juz21o/page_1.jpg']
2024-08-25 14:19:53,153 - INFO - Experiment 0: Processing file 20120424_Pillsbury Winthrop Shaw Pittman, LLP_Amendment_Amendment.pdf


Converted and resized image saved to /tmp/tmpn1juz21o/page_1.jpg with final quality 85
Converted PDF /tmp/tmpn1juz21o/19830401_Bermuda Tourism Authority_Amendment_Amendment.pdf to images: ['/tmp/tmpn1juz21o/page_1.jpg']


2024-08-25 14:19:53,904 - INFO - Converted and resized image saved to /tmp/tmp4l34cy1b/page_1.jpg with final quality 85


...

Converted and resized image saved to /tmp/tmppuiwgpmi/page_3.jpg with final quality 85
Converted PDF /tmp/tmppuiwgpmi/20110912_BLJ Worldwide LTD_Smrikarov, Marc Philip George_Short-Form.pdf to images: ['/tmp/tmppuiwgpmi/page_1.jpg', '/tmp/tmppuiwgpmi/page_2.jpg', '/tmp/tmppuiwgpmi/page_3.jpg']


2024-08-25 14:23:53,471 - INFO - Converted and resized image saved to /tmp/tmp1zn79hhs/page_1.jpg with final quality 85


Converted and resized image saved to /tmp/tmp1zn79hhs/page_1.jpg with final quality 85


2024-08-25 14:23:53,775 - INFO - Converted and resized image saved to /tmp/tmp1zn79hhs/page_2.jpg with final quality 85
2024-08-25 14:23:53,776 - INFO - Converted PDF /tmp/tmp1zn79hhs/20090409_Daniel J. Edelman, Inc._Segall, Jules Peter_Short-Form.pdf to images: ['/tmp/tmp1zn79hhs/page_1.jpg', '/tmp/tmp1zn79hhs/page_2.jpg']


Converted and resized image saved to /tmp/tmp1zn79hhs/page_2.jpg with final quality 85
Converted PDF /tmp/tmp1zn79hhs/20090409_Daniel J. Edelman, Inc._Segall, Jules Peter_Short-Form.pdf to images: ['/tmp/tmp1zn79hhs/page_1.jpg', '/tmp/tmp1zn79hhs/page_2.jpg']
Experiment 1 completed successfully.


In [9]:
batch_file_paths = split_jsonl_file(output_file, max_size=209715200, max_lines=1000)  # Adjust max_lines as needed
batch_ids = send_batch_requests(batch_file_paths)
print(f"Batch IDs: {batch_ids}")

2024-08-25 14:46:24,435 - INFO - HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"
2024-08-25 14:46:25,264 - INFO - HTTP Request: POST https://api.openai.com/v1/batches "HTTP/1.1 200 OK"
2024-08-25 14:46:31,340 - INFO - HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"
2024-08-25 14:46:31,890 - INFO - HTTP Request: POST https://api.openai.com/v1/batches "HTTP/1.1 200 OK"


Batch IDs: ['batch_KSqKx3BsQwf7m3kpbKbrb4Qz', 'batch_QlSG3OlW737tMQBIVUWunIL2']


In [6]:
if __name__ == "__main__":
    all_filename_mappings, all_output_data = load_experiment_results(base_output_directory)
    print("DONE")


DONE


In [7]:
ground_truth_annotations, _ = load_dataset(dataset_path)
logging.info("Loaded ground truth annotations")
response_dir = os.path.join(base_output_directory, "responses")


retrieve_and_process_batch_results(response_dir, ground_truth_annotations, all_filename_mappings, all_output_data, base_output_directory)

2025-02-10 11:09:25,311 - INFO - Loaded dataset from vrdu/registration-form/main/dataset.jsonl.gz
2025-02-10 11:09:25,313 - INFO - Loaded ground truth annotations
2025-02-10 11:09:25,318 - INFO - Processing annotations for file: 19830401_Bermuda Tourism Authority_Amendment_Amendment.pdf
2025-02-10 11:09:25,321 - INFO - Processed and saved reconciled predictions for 19830401_Bermuda Tourism Authority_Amendment_Amendment.pdf
2025-02-10 11:09:25,322 - INFO - Processing annotations for file: 20120424_Pillsbury Winthrop Shaw Pittman, LLP_Amendment_Amendment.pdf
2025-02-10 11:09:25,324 - INFO - Processed and saved reconciled predictions for 20120424_Pillsbury Winthrop Shaw Pittman, LLP_Amendment_Amendment.pdf
2025-02-10 11:09:25,325 - INFO - Processing annotations for file: 19920601_JETRO, San Francisco_Amendment_Amendment.pdf
2025-02-10 11:09:25,327 - INFO - Processed and saved reconciled predictions for 19920601_JETRO, San Francisco_Amendment_Amendment.pdf
2025-02-10 11:09:25,328 - INFO - 

Loaded dataset from vrdu/registration-form/main/dataset.jsonl.gz

RAW PRED: [{'file_date': 'Apr 20, 1983', 'foreign_principle_name': '', 'registrant_name': 'Bermuda Department of Tourism', 'registration_num': '430', 'signer_name': 'Ronald N. Bassett', 'signer_title': 'General Manager, North America'}]

FLAT PRED:  [{'file_date': 'Apr 20, 1983', 'foreign_principle_name': '', 'registrant_name': 'Bermuda Department of Tourism', 'registration_num': '430', 'signer_name': 'Ronald N. Bassett', 'signer_title': 'General Manager, North America'}]

Reconciled PRED:  {'file_date': 'Apr 20, 1983', 'foreign_principle_name': '', 'registrant_name': 'Bermuda Department of Tourism', 'registration_num': '430', 'signer_name': 'Ronald N. Bassett', 'signer_title': 'General Manager, North America'}
SANITIZED FİLENAME:  19830401_Bermuda_Tourism_Authority_Amendment_Amendmentpdf
ORG FİLENAME BEFORE:  19830401_Bermuda Tourism Authority_Amendment_Amendment.pdf
ORG FİLENAME AFTER:  19830401_Bermuda Tourism Authori

2025-02-10 11:09:25,513 - INFO - Processing annotations for file: 19930601_International Registries, Inc._Dissemination Report_Dissemination Report.pdf
2025-02-10 11:09:25,536 - INFO - Processed and saved reconciled predictions for 19930601_International Registries, Inc._Dissemination Report_Dissemination Report.pdf
2025-02-10 11:09:25,538 - INFO - Processing annotations for file: 19930901_Scottish Enterprise_Dissemination Report_Dissemination Report.pdf
2025-02-10 11:09:25,551 - INFO - Processed and saved reconciled predictions for 19930901_Scottish Enterprise_Dissemination Report_Dissemination Report.pdf
2025-02-10 11:09:25,552 - INFO - Processing annotations for file: 19920701_Daniel J. Edelman, Inc._Dissemination Report_Dissemination Report.pdf
2025-02-10 11:09:25,556 - INFO - Processed and saved reconciled predictions for 19920701_Daniel J. Edelman, Inc._Dissemination Report_Dissemination Report.pdf
2025-02-10 11:09:25,563 - INFO - Processing annotations for file: 19950619_JETRO, 

GT:  {'registration_num': ['4533'], 'registrant_name': ['International Registries, Inc.'], 'signer_title': ['President'], 'file_date': ['June 7,\n1993'], 'signer_name': ['A.N. Stewart']}

initial_pred: {'file_date': ('93 JUL 10 AM 11:34', 'June 7, 1993'), 'foreign_principle_name': '', 'registrant_name': 'International Registries, Inc.', 'registration_num': '4533', 'signer_name': 'A.N. Stewart', 'signer_title': 'President'}

mapped_pred: {'file_date': ('93 JUL 10 AM 11:34', 'June 7, 1993'), 'foreign_principle_name': '', 'registrant_name': 'International Registries, Inc.', 'registration_num': '4533', 'signer_name': 'A.N. Stewart', 'signer_title': 'President'}

cleaned_pred: {'file_date': ('1993-07-10', '1993-06-07'), 'foreign_principle_name': '', 'registrant_name': 'International Registries, Inc.', 'registration_num': '4533', 'signer_name': 'A.n. Stewart', 'signer_title': 'president'} 

ground_truth_value: {'file_date': ['June 7,\n1993'], 'foreign_principle_name': '', 'registrant_name': 

2025-02-10 11:09:25,753 - INFO - Processing annotations for file: 20160622_Raben Group, LLC_Raben, Robert_Short-Form.pdf
2025-02-10 11:09:25,764 - INFO - Processed and saved reconciled predictions for 20160622_Raben Group, LLC_Raben, Robert_Short-Form.pdf
2025-02-10 11:09:25,767 - INFO - Processing annotations for file: 20100909_Gephardt Group Government Affairs, LLC_Messmer, Michael_Short-Form.pdf
2025-02-10 11:09:25,772 - INFO - Processed and saved reconciled predictions for 20100909_Gephardt Group Government Affairs, LLC_Messmer, Michael_Short-Form.pdf
2025-02-10 11:09:25,773 - INFO - Processing annotations for file: 20100111_O_Brien Gentry _ Scott, LLC_Gentry, James_Short-Form.pdf
2025-02-10 11:09:25,783 - INFO - Processed and saved reconciled predictions for 20100111_O_Brien Gentry _ Scott, LLC_Gentry, James_Short-Form.pdf
2025-02-10 11:09:25,784 - INFO - Processing annotations for file: 20170130_Burson-Marsteller, LLC (Washington, DC)_Pruitt, David_Short-Form.pdf
2025-02-10 11:09

...

2025-02-10 11:09:25,976 - ERROR - Invalid JSON format in message_content for custom_id 1_UTL_Amendment_19950929_Hogan_Lovells_US_LLP_Amendment_Amendmentpdf_d31f134433904b97afdd02f42c9075d6: {
    "file_date": "29th day of September, 1945",
    "foreign_principle_name": "",
    "registrant_name": "",
    "registration_num": "",
    "signer_name": "Jack S. Powell",
    "signer_title": "",
}
2025-02-10 11:09:25,980 - INFO - Processing annotations for file: 19830401_Bermuda Tourism Authority_Amendment_Amendment.pdf
2025-02-10 11:09:25,982 - INFO - Processed and saved reconciled predictions for 19830401_Bermuda Tourism Authority_Amendment_Amendment.pdf
2025-02-10 11:09:25,983 - INFO - Processing annotations for file: 20120424_Pillsbury Winthrop Shaw Pittman, LLP_Amendment_Amendment.pdf
2025-02-10 11:09:25,985 - INFO - Processed and saved reconciled predictions for 20120424_Pillsbury Winthrop Shaw Pittman, LLP_Amendment_Amendment.pdf
2025-02-10 11:09:25,986 - INFO - Processing annotations fo


RAW PRED: [{'file_date': 'Apr 20, 1983', 'foreign_principle_name': '', 'registrant_name': 'Bermuda Department of Tourism', 'registration_num': '430', 'signer_name': 'Ronald N. Bassett', 'signer_title': 'General Manager, North America'}]

FLAT PRED:  [{'file_date': 'Apr 20, 1983', 'foreign_principle_name': '', 'registrant_name': 'Bermuda Department of Tourism', 'registration_num': '430', 'signer_name': 'Ronald N. Bassett', 'signer_title': 'General Manager, North America'}]

Reconciled PRED:  {'file_date': 'Apr 20, 1983', 'foreign_principle_name': '', 'registrant_name': 'Bermuda Department of Tourism', 'registration_num': '430', 'signer_name': 'Ronald N. Bassett', 'signer_title': 'General Manager, North America'}
SANITIZED FİLENAME:  19830401_Bermuda_Tourism_Authority_Amendment_Amendmentpdf
ORG FİLENAME BEFORE:  19830401_Bermuda Tourism Authority_Amendment_Amendment.pdf
ORG FİLENAME AFTER:  19830401_Bermuda Tourism Authority_Amendment_Amendment.pdf
GT:  {'registration_num': ['430'], 're

2025-02-10 11:09:26,195 - INFO - Processing annotations for file: 19920301_JETRO, Los Angeles_Dissemination Report_Dissemination Report.pdf
2025-02-10 11:09:26,208 - INFO - Processed and saved reconciled predictions for 19920301_JETRO, Los Angeles_Dissemination Report_Dissemination Report.pdf
2025-02-10 11:09:26,210 - INFO - Processing annotations for file: 19930101_JETRO, Chicago_Dissemination Report_Dissemination Report.pdf
2025-02-10 11:09:26,215 - INFO - Processed and saved reconciled predictions for 19930101_JETRO, Chicago_Dissemination Report_Dissemination Report.pdf
2025-02-10 11:09:26,219 - INFO - Processing annotations for file: 19910201_World Zionist Organization - American Section, Inc._Dissemination Report_Dissemination Report.pdf
2025-02-10 11:09:26,224 - INFO - Processed and saved reconciled predictions for 19910201_World Zionist Organization - American Section, Inc._Dissemination Report_Dissemination Report.pdf
2025-02-10 11:09:26,225 - INFO - Processing annotations for 


RAW PRED: [{'file_date': '', 'foreign_principle_name': 'JAPAN External Trade Organization Tokyo, Japan', 'registrant_name': 'JETRO, Los Angeles', 'registration_num': '1833', 'signer_name': '', 'signer_title': ''}, {'file_date': '3/27/92', 'foreign_principle_name': '', 'registrant_name': '', 'registration_num': '', 'signer_name': 'Eiichi Nagata', 'signer_title': 'Executive Director'}]

FLAT PRED:  [{'file_date': '', 'foreign_principle_name': 'JAPAN External Trade Organization Tokyo, Japan', 'registrant_name': 'JETRO, Los Angeles', 'registration_num': '1833', 'signer_name': '', 'signer_title': ''}, {'file_date': '3/27/92', 'foreign_principle_name': '', 'registrant_name': '', 'registration_num': '', 'signer_name': 'Eiichi Nagata', 'signer_title': 'Executive Director'}]

Reconciled PRED:  {'file_date': '3/27/92', 'foreign_principle_name': 'JAPAN External Trade Organization Tokyo, Japan', 'registrant_name': 'JETRO, Los Angeles', 'registration_num': '1833', 'signer_name': 'Eiichi Nagata', '

2025-02-10 11:09:26,403 - INFO - Processed and saved reconciled predictions for 20160622_Raben Group, LLC_Raben, Robert_Short-Form.pdf
2025-02-10 11:09:26,421 - INFO - Processing annotations for file: 20100909_Gephardt Group Government Affairs, LLC_Messmer, Michael_Short-Form.pdf
2025-02-10 11:09:26,428 - INFO - Processed and saved reconciled predictions for 20100909_Gephardt Group Government Affairs, LLC_Messmer, Michael_Short-Form.pdf
2025-02-10 11:09:26,429 - INFO - Processing annotations for file: 20100111_O_Brien Gentry _ Scott, LLC_Gentry, James_Short-Form.pdf
2025-02-10 11:09:26,437 - INFO - Processed and saved reconciled predictions for 20100111_O_Brien Gentry _ Scott, LLC_Gentry, James_Short-Form.pdf
2025-02-10 11:09:26,438 - INFO - Processing annotations for file: 20170130_Burson-Marsteller, LLC (Washington, DC)_Pruitt, David_Short-Form.pdf
2025-02-10 11:09:26,442 - INFO - Processed and saved reconciled predictions for 20170130_Burson-Marsteller, LLC (Washington, DC)_Pruitt, 